# Paris Landing Page

### Example with static content

- [ ] show assets images
- [ ] show assets file browser

### Command line
```python
conda run -n status38v1 panel serve --address localhost --port 55555 --prefix=/services/paris/ app.ipynb --allow-websocket-origin=someserver.uni-freiburg.de --static-dirs assets='./assets' --autoreload
```

In [ ]:
import datetime
import glob
import io
import itertools
import json
import logging
import os
import pickle
import shutil
import subprocess
import sys
from pathlib import Path
from urllib.parse import urljoin

import humanize

# data science
import numpy as np
import pandas as pd

# dynamic web interface
import panel as pn
import param

# plotting libraries
import plotly.express as px

In [ ]:
if not "main_app" in locals():
    main_app = True

In [ ]:
logging.info("Run as panel app: {}".format(main_app))

### Prepare visualisation

In [ ]:
def someserver_footer_markdown():

    txt = """
        
        2022<br>
        [Professur für Umweltmeteorologie](https://meteo.uni-freiburg.de)<br>
        [Fakultät für Umwelt und Natürliche Ressourcen](https://www.unr.uni-freiburg.de/)<br>
        [Albert-Ludwigs-Universität Freiburg](https://www.uni-freiburg.de/)<br><br>
        [Impressum](https://uni-freiburg.de/impressum)

        <sup>
        Die hier gezeigten Daten werden automatisch erfasst und ausschließlich für Zwecke der Forschung, Studium und Lehre von der Albert-Ludwigs-Universität Freiburg 
        zur Verfügung gestellt. Es besteht kein rechtlicher Anspruch auf dauerhafte Verfügbarkeit dieser Daten. Weiter übernimmt die Albert-Ludwigs-Universität Freiburg 
        keine Gewähr für die Richtigkeit, Vollständigkeit und Aktualität der bereitgestellten Informationen. Die Nutzung aller hier gezeigten und geladenen Daten erfolgt 
        auf eigene Gefahr des Nutzers. Die Albert-Ludwigs-Universität Freiburg übernimmt keine Haftung für Schäden materieller oder ideeller Art, die aufgrund der 
        Nutzung oder Nichtnutzung bzw. durch die Nutzung fehlerhafter und unvollständiger Informationen entstehen. Die gesetzlichen Bestimmungen bleiben unberührt.
        </sup>
        """
    return txt

def status_urbisphere_footer_markdown():

    txt = """
        
        <b><i>urbisphere</i> resource:</b> <br><br>
        The <i>urbisphere</i><br>
        <b>Data Management Policy</b> (DMP),<br>
        <b>Incidental Findings Policy</b> (IFP) and<br> 
        <b>Publication Code of Conduct</b> (PCC)          
        apply."""
    return txt

In [ ]:
class someserver_paris(param.Parameterized):

    # init config, best read from a JSON or TOML file.
    app_config = {
        "date_range": [
            pd.to_datetime("2022-10-01"),
            datetime.datetime.now(),
        ],
        "startpage": [
            {
                "title": "File Availability Dashboard",
                "description": "Monitor the status of data ingestion and data management infrastructure.",
                "url": "https://someserver.uni-freiburg.de/services/d/",
            },            
            {
                "title": "JupyterLab Notebook",
                "description": "IDE for R or Python.",
                "url": "https://someserver.uni-freiburg.de/services/p/",
            },
            {
                "title": "JupyterLab Notebook (Education)",
                "description": "IDE for R or Python for students and partners.",
                "url": "https://someserver.uni-freiburg.de/services/e/",
            },            
        ],
        "assets": { 
            "DWL": [
            {
                "img": {
                    "type": "png",
                    "file_list": ["assets/folder1/*.png"],
                    "embed": False,
                    "url_base": "https://someserver.uni-freiburg.de/services/paris/",
                },
            }],
            "ACL": [{
                "img": {
                    "type": "png",
                    "file_list": ["assets/folder2/*.png"],
                    "embed": False,
                    "url_base": "https://someserver.uni-freiburg.de/services/paris/",
                },
            },            
        ]
        }
    }

    # example selector
    daterange = param.DateRange(
        tuple(app_config["date_range"]), bounds=tuple(app_config["date_range"])
    )

    def assets(self):
        ad = {}
        for k, assets in self.app_config["assets"].items():
            if not k in ad:
                ad[k] = {'png':[]}
            for n in assets:
                if "img" in n:
                    if n["img"]["type"] in ["png"]:
                        fns = [
                            item
                            for sublist in [
                                glob.glob(item) for item in n["img"]["file_list"]
                            ]
                            for item in sublist
                        ]
                        for fn in fns:
                            img_args = dict(embed=n["img"]["embed"], object=fn)
                            ad[k]["png"].append(img_args)
                            logging.debug("Asset added for '%s'", fn)
        return ad

    def startpage(self):
        md = ""
        for n in self.app_config["startpage"]:
            md += "##### {}\n".format(n["title"])
            md += " - Description: {}\n".format(n["description"]) if n["description"] != "" else ""
            md += " - URL: [{}]({})\n".format(n["url"], n["url"])
            logging.debug("Startpage entry added '%s'", n["title"])
        return md

In [ ]:
def someserver_paris_template():

    # define the panes
    pn.extension()
    pn.extension(sizing_mode="stretch_width")

    controls = pn.Param(
        obj.param,
        widgets={
            "daterange": {
                "widget_type": pn.widgets.DatetimeRangePicker,
                "enable_time": False,
                "enable_seconds": False,
            },
        },
    )

    pn.GridSpec(width=800, height=600)

    # Prepare Panes
    pane0_0 = pn.pane.Markdown("""## Links""")
    pane0_1 = pn.pane.Markdown(obj.startpage())

    pane1_0 = pn.pane.Markdown("""## Assets""")
    pane1_1 = pn.Column(
        *[pn.pane.PNG(width=900, **args) for args in obj.assets()["ACL"]['png']]
    )

    pane2_0 = pn.pane.Markdown("""## Assets""")
    pane2_1 = pn.Column(
        *[pn.pane.PNG(width=900, **args) for args in obj.assets()["DWL"]['png']]
    )    

    footer1 = pn.pane.Alert(status_urbisphere_footer_markdown(), alert_type="dark")
    footer2 = pn.pane.Alert(someserver_footer_markdown(), alert_type="dark")    

    # framework
    bootstrap = pn.template.BootstrapTemplate(title="status.meteo / services / paris")
    bootstrap.header_background = "#666666"

    # controls
    bootstrap.sidebar.append(
        pn.Column(
            controls,
            pn.layout.VSpacer(),
            footer1,
            footer2,
            pn.layout.VSpacer(),
        )
    )

    # add Panes and Widgets
    bootstrap.main.append(
        pn.Tabs(
            (
                "Startpage",
                pn.Column(
                    pane0_0,
                    pane0_1,
                    sizing_mode="stretch_width",
                ),
            ),
            (
                "ACL",
                pn.Column(
                    pane1_0,
                    pane1_1,
                    sizing_mode="stretch_width",                    
                ),
            ),
            (
                "DWL",
                pn.Column(
                    pane2_0,
                    pane2_1,
                    sizing_mode="stretch_width",                    
                ),
            ),            
            dynamic=True,
            active=0,
        )
    )

    # add logo?
    bootstrap.sidebar.append(
        pn.pane.SVG("http://weather.uni-freiburg.de/img/Uni_Logo.svg", width=75)
    )

    return bootstrap

In [ ]:
# cs.stop()
if main_app:
    obj = someserver_paris(name="Paris Assets Control")
    template = someserver_paris_template()
    cs = template.servable()